# Central Alps download: Tyrol, Lombardia

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# Preamble

import logging
logging.basicConfig(level=logging.DEBUG)

from dataclasses import astuple
from urllib.request import urlretrieve
import os
import sys

CURDIR = os.path.realpath(os.curdir)
# sys.path.append(os.path.join(CURDIR, 'src'))
from src.bbox import BBox
from src.gdal_slope_util import check_run

## Austria: Tyrol

In [2]:
!mkdir data/tyrol
!mkdir -p data/dtm_final
url = 'https://gis.tirol.gv.at/ogd/geografie_planung/DGM/DGM_Tirol_5m_epsg31254.zip'
urlretrieve(url, 'data/tyrol/DGM_Tirol_5m_epsg31254.zip')

('data/tyrol/DGM_Tirol_5m_epsg31254.zip',
 <http.client.HTTPMessage at 0x7fd11d0fe490>)

In [4]:
!unzip data/tyrol/DGM_Tirol_5m_epsg31254.zip -d data/dtm_final


Archive:  data/tyrol/DGM_Tirol_5m_epsg31254.zip
  inflating: DGM_Tirol_5m_epsg31254_2006_2019.tif  


In [8]:
!gdalinfo DGM_Tirol_5m_epsg31254_2006_2019.tif | grep -v '^    '

Driver: GTiff/GeoTIFF
Files: DGM_Tirol_5m_epsg31254_2006_2019.tif
Size is 44724, 26001
Coordinate System is:
PROJCRS["MGI / Austria GK West",
Data axis to CRS axis mapping: 2,1
Origin = (-18617.500000000000000,295002.500000000000000)
Pixel Size = (5.000000000000000,-5.000000000000000)
Metadata:
  AREA_OR_POINT=Area
  DataType=Generic
Image Structure Metadata:
  COMPRESSION=DEFLATE
  INTERLEAVE=BAND
Corner Coordinates:
Upper Left  (  -18617.500,  295002.500) ( 10d 5' 5.32"E, 47d47'38.06"N)
Lower Left  (  -18617.500,  164997.500) ( 10d 5'24.77"E, 46d37'27.88"N)
Upper Right (  205002.500,  295002.500) ( 13d 4' 5.80"E, 47d45'41.69"N)
Lower Right (  205002.500,  164997.500) ( 13d 0'32.03"E, 46d35'36.15"N)
Center      (   93192.500,  230000.000) ( 11d33'48.48"E, 47d12'10.26"N)
Band 1 Block=256x256 Type=Float32, ColorInterp=Gray
  Min=460.346 Max=3796.002 
  Minimum=460.346, Maximum=3796.002, Mean=1750.300, StdDev=652.179
  NoData Value=3.39999995214436425e+38
  Overviews: 14908x8667, 4970x28

## Italy: Lombardia

Download from [geoportale.regione.lombardia.it](https://www.geoportale.regione.lombardia.it/download-pacchetti?p_p_id=dwnpackageportlet_WAR_gptdownloadportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_dwnpackageportlet_WAR_gptdownloadportlet_metadataid=%7BFC06681A-2403-481F-B6FE-5F952DD48BAF%7D)

In [4]:
%cd $CURDIR
!mkdir -p data/lombardia/dwn
%cd data/lombardia/dwn

/home/me/code/eddy-geek/eslope/development
/home/me/code/eddy-geek/slope-ign-alti/lombardia/dwn


In [7]:
LBURL = 'https://www.geoportale.regione.lombardia.it/rlregis_download/service/package?dbId={}&cod={}'
targets  = []
# Cities
for dbid, cod, name in [
            (313, 16, 'Bergamo'),
            # (313, 19, 'Cremona'),
            (313, 97, 'Lecco'  ),
            # (313, 98, 'Lodi'   ),
            # (313, 20, 'Mantova'),
            # (313, 15, 'Milan'  ),
            (313, 108, 'Monza' ),
            # (313, 18, 'Pavia'  ),
            (313, 14, 'Sondrio'),
            (313, 12, 'Varese' )]:
    target = f'lombardia_{name}_dtm.zip'
    targets.append(target)
    urlretrieve(LBURL.format(dbid, cod), target)

# Mountains
for zone in range(1, 24):
    z = 'ZO' + str(zone).rjust(2, '0')
    url = LBURL.format(321, z)
    target = f'lombardia_{z}_dtm.zip'
    targets.append(target)
    urlretrieve(url, f'lombardia_{z}_dtm.zip')

# ~ 3 minutes


In [10]:
targets

['lombardia_Bergamo_dtm.zip',
 'lombardia_Lecco_dtm.zip',
 'lombardia_Monza_dtm.zip',
 'lombardia_Sondrio_dtm.zip',
 'lombardia_Varese_dtm.zip',
 'lombardia_ZO01_dtm.zip',
 'lombardia_ZO02_dtm.zip',
 'lombardia_ZO03_dtm.zip',
 'lombardia_ZO04_dtm.zip',
 'lombardia_ZO05_dtm.zip',
 'lombardia_ZO06_dtm.zip',
 'lombardia_ZO07_dtm.zip',
 'lombardia_ZO08_dtm.zip',
 'lombardia_ZO09_dtm.zip',
 'lombardia_ZO10_dtm.zip',
 'lombardia_ZO11_dtm.zip',
 'lombardia_ZO12_dtm.zip',
 'lombardia_ZO13_dtm.zip',
 'lombardia_ZO14_dtm.zip',
 'lombardia_ZO15_dtm.zip',
 'lombardia_ZO16_dtm.zip',
 'lombardia_ZO17_dtm.zip',
 'lombardia_ZO18_dtm.zip',
 'lombardia_ZO19_dtm.zip',
 'lombardia_ZO20_dtm.zip',
 'lombardia_ZO21_dtm.zip',
 'lombardia_ZO22_dtm.zip',
 'lombardia_ZO23_dtm.zip']

In [12]:
!CPL_ZIP_ENCODING=UTF-8 gdalinfo '/vsizip/lombardia_ZO06_dtm.zip/DTM5_CM_SEBINO BRESCIANO.IMG' |grep -v '^    '

Driver: HFA/Erdas Imagine Images (.img)
Files: /vsizip/lombardia_ZO06_dtm.zip/DTM5_CM_SEBINO BRESCIANO.IMG
Size is 3659, 4994
Coordinate System is:
PROJCRS["WGS_1984_UTM_Zone_32N",
Data axis to CRS axis mapping: 1,2
Origin = (575851.869400000199676,5076326.248099999502301)
Pixel Size = (5.000000000000000,-5.000000000000000)
Metadata:
  DataType=Generic
Corner Coordinates:
Upper Left  (  575851.869, 5076326.248) (  9d58'36.17"E, 45d50'10.77"N)
Lower Left  (  575851.869, 5051356.248) (  9d58'22.10"E, 45d36'41.79"N)
Upper Right (  594146.869, 5076326.248) ( 10d12'44.08"E, 45d50' 2.65"N)
Lower Right (  594146.869, 5051356.248) ( 10d12'26.62"E, 45d36'33.73"N)
Center      (  584999.369, 5063841.248) ( 10d 5'32.23"E, 45d43'22.46"N)
Band 1 Block=64x64 Type=Float32, ColorInterp=Undefined
  Description = Layer_1
  Min=181.640 Max=1950.000 
  Minimum=181.640, Maximum=1950.000, Mean=615.285, StdDev=433.963
  NoData Value=-3.4028234663852886e+38
  Overviews: 1830x2497, 915x1249, 458x625, 229x313
  

In [ ]:
zfor f in DTM0508_002_000{001..914} ; do echo "/vsizip/{aoste_dtm_zips/$f.zip}/$f.ASC" >>! aoste_dtm_zips.txt ; done